In [1]:
import pandas as pd
import numpy as np
import tqdm

In [2]:
#last, first, min, max,std,mean,median
importnant_columns = ['customer_ID', 'S_2', 'P_2', 'P_3', 'D_39', 'D_42', 'D_43', 'D_44', 'D_46', 'D_48',
                      'D_49', 'D_62', 'D_77','B_2', 'B_3', 'B_4', 'B_8', 'B_9', 'B_11','B_33','R_1', 'S_3',
                     "B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68",
                     ]# 'target']

In [3]:
train = pd.read_pickle(r'C:\Users\domin\Downloads\AE_pickled\test.pkl')[importnant_columns]
stat_list = ['first', 'last', 'median', 'mean', 'std', 'max']
#test = pd.read_pickle(r'C:\Users\domin\Downloads\AE_pickled\test.pkl')

In [4]:
def number_cat(list_of_categories):
    """Function to cast categorical vatriables as numeric, output - dictionary of mappings"""
    unique_values = list_of_categories.values.unique()
    corresponding_map = {}
    i = 0
    for q in unique_values:
        corresponding_map[q] = i
        i +=1
    return corresponding_map


In [5]:
mapping = number_cat(train['D_63'])
train['D_63'] = [mapping[q] for q in train['D_63']]
mapping = number_cat(train['D_64'])
train['D_64'] = [mapping[q] for q in train['D_64']]

In [6]:
def process_and_feature_engineer(df):
    # FEATURE ENGINEERING FROM 
    # https://www.kaggle.com/code/huseyincot/amex-agg-data-how-it-created
    all_cols = [c for c in list(df.columns) if c not in ['customer_ID','S_2']]
    cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
    num_features = ['P_2', 'P_3', 'D_39', 'D_42', 'D_43', 'D_44', 'D_46', 'D_48', 'D_49', 'D_62', 'D_77','B_2', 'B_3', 'B_4', 'B_8', 'B_9', 'B_11','B_33','R_1', 'S_3']

    test_num_agg = df.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]

    test_cat_agg = df.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
    df = pd.concat([test_num_agg, test_cat_agg], axis=1)
    del test_num_agg, test_cat_agg
    print('shape after engineering', df.shape )
    print(df.columns)
    return df
train_processed = process_and_feature_engineer(train)

shape after engineering (924621, 133)
Index(['P_2_mean', 'P_2_std', 'P_2_min', 'P_2_max', 'P_2_last', 'P_3_mean',
       'P_3_std', 'P_3_min', 'P_3_max', 'P_3_last',
       ...
       'D_63_nunique', 'D_64_count', 'D_64_last', 'D_64_nunique', 'D_66_count',
       'D_66_last', 'D_66_nunique', 'D_68_count', 'D_68_last', 'D_68_nunique'],
      dtype='object', length=133)


In [7]:
for col in train_processed.columns: print(col)

P_2_mean
P_2_std
P_2_min
P_2_max
P_2_last
P_3_mean
P_3_std
P_3_min
P_3_max
P_3_last
D_39_mean
D_39_std
D_39_min
D_39_max
D_39_last
D_42_mean
D_42_std
D_42_min
D_42_max
D_42_last
D_43_mean
D_43_std
D_43_min
D_43_max
D_43_last
D_44_mean
D_44_std
D_44_min
D_44_max
D_44_last
D_46_mean
D_46_std
D_46_min
D_46_max
D_46_last
D_48_mean
D_48_std
D_48_min
D_48_max
D_48_last
D_49_mean
D_49_std
D_49_min
D_49_max
D_49_last
D_62_mean
D_62_std
D_62_min
D_62_max
D_62_last
D_77_mean
D_77_std
D_77_min
D_77_max
D_77_last
B_2_mean
B_2_std
B_2_min
B_2_max
B_2_last
B_3_mean
B_3_std
B_3_min
B_3_max
B_3_last
B_4_mean
B_4_std
B_4_min
B_4_max
B_4_last
B_8_mean
B_8_std
B_8_min
B_8_max
B_8_last
B_9_mean
B_9_std
B_9_min
B_9_max
B_9_last
B_11_mean
B_11_std
B_11_min
B_11_max
B_11_last
B_33_mean
B_33_std
B_33_min
B_33_max
B_33_last
R_1_mean
R_1_std
R_1_min
R_1_max
R_1_last
S_3_mean
S_3_std
S_3_min
S_3_max
S_3_last
B_30_count
B_30_last
B_30_nunique
B_38_count
B_38_last
B_38_nunique
D_114_count
D_114_last
D_114_nunique


In [19]:
targets = train[['customer_ID', 'target']].drop_duplicates().set_index('customer_ID')
targets

,target
customer_ID,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0
...,...
ffff41c8a52833b56430603969b9ca48d208e7c192c6a4081a6acc28cf4f8af7,0
ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fdd3e5b57cfcbee30286,0
ffff9984b999fccb2b6127635ed0736dda94e544e67e026eee4d20f680639ff6,0


In [20]:
train_processed = train_processed.merge(targets, how = 'left', left_index = True, right_index = True)

In [8]:
train_processed

,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,P_3_mean,P_3_std,P_3_min,P_3_max,P_3_last,...,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique
customer_ID,,,,,,,,,,,,,,,,,,,,,
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0.601562,0.020198,0.568848,0.631348,0.568848,0.591797,0.043778,0.535156,0.634277,0.560547,...,1,9,1,2,0,NaN,0,5,6.0,1
00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5,0.862305,0.031432,0.794434,0.913574,0.841309,0.553223,0.082924,0.385742,0.653809,0.525879,...,1,13,2,1,0,NaN,0,13,6.0,1
0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8,0.749023,0.061435,0.673340,0.834961,0.697754,0.670898,0.191139,0.381592,0.944336,0.566406,...,1,13,1,2,13,1.0,1,13,4.0,2
00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694,0.474609,0.028883,0.428467,0.514160,0.513184,0.610840,0.087533,0.464844,0.794434,0.628418,...,1,13,3,1,0,NaN,0,13,5.0,1
00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a4693dd914fca22557,0.323975,0.049886,0.254395,0.425781,0.254395,0.634766,0.126156,0.467529,0.877441,0.569824,...,1,13,3,2,0,NaN,0,13,5.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c5d60460dba6dedc41e,0.631836,0.015766,0.597656,0.646973,0.646973,0.629395,0.049971,0.556641,0.757812,0.662109,...,1,13,2,1,0,NaN,0,13,6.0,1
ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3a4f0ca3de613b0b2ad,0.546875,0.053264,0.471191,0.634766,0.471191,0.599121,0.118953,0.443359,0.846191,0.479004,...,1,13,2,1,0,NaN,0,13,6.0,2
ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475cb095d2443a68030f1,0.239014,0.048400,0.195679,0.316650,0.206421,0.583496,0.117093,0.405273,0.729980,0.598633,...,1,13,2,1,0,NaN,0,13,6.0,1


In [9]:
len(train_processed)

924621

In [10]:
train_processed.to_pickle('test_correct.pkl')